Here I'm going to attempt make a table for the paper based on the parameter files. 

In [2]:
import os
os.environ['XLA_PYTHON_CLIENT_MEM_FRACTION'] = '0.30'

import numpy as np



In [3]:
x0_f300m = np.load("hg3fit_F300M_m_stars_bounded_quad_new.npz.npy")
x0_f360m = np.load("hg3fit_F360M_full_params.npy")
n_nodes = 6 #Number of parameters associated with the scattering phase function. 

In [4]:
cent = [173.4,149.1]

### PSF Center Parameters: 
f300m_roll1_centers = [x0_f300m[18+4*n_nodes]+cent[0],x0_f300m[19+4*n_nodes]+cent[1]] #[x,y] 
f300m_roll2_centers = [x0_f300m[20+4*n_nodes]+cent[0],x0_f300m[21+4*n_nodes]+cent[1]] #[x,y]

f360m_roll1_centers = [x0_f360m[18+4*n_nodes]+cent[0],x0_f360m[19+4*n_nodes]+cent[1]] #[x,y]
f360m_roll2_centers = [x0_f360m[20+4*n_nodes]+cent[0],x0_f360m[21+4*n_nodes]+cent[1]] #[x,y]

### Commonon Parameters: 
inc = np.degrees(x0_f300m[0])
pa = np.degrees(x0_f300m[1])

In [5]:
############# SMA ##############
#Parameters from October
a0s= [40.0,220.0,510.0,330.0] #r_c same for both disks
f0_f300m = [3e4,7.5e3,140,410] #n0_scale
f0_f360m = [1.5e5,2.5e3,140,410] #n0_scale

#Rescale x0_f300m by these values, where appropriate
for i in range(len(a0s)):
    x0_f300m[2+10*i] *= f0_f300m[i]
    x0_f300m[3+10*i] *= a0s[i]
    x0_f360m[2+10*i] *= f0_f360m[i]
    x0_f360m[3+10*i] *= a0s[i]


In [6]:
#Normalize the weights
x0_f300m[9:12] /= np.sum(x0_f300m[9:12])
x0_f300m[13+n_nodes:16+n_nodes] /= np.sum(x0_f300m[13+n_nodes:16+n_nodes])
x0_f300m[17+2*n_nodes:20+2*n_nodes] /= np.sum(x0_f300m[17+2*n_nodes:20+2*n_nodes])
x0_f300m[21+3*n_nodes:24+3*n_nodes] /= np.sum(x0_f300m[21+3*n_nodes:24+3*n_nodes])

x0_f360m[9:12] /= np.sum(x0_f360m[9:12])
x0_f360m[13+n_nodes:16+n_nodes] /= np.sum(x0_f360m[13+n_nodes:16+n_nodes])
x0_f360m[17+2*n_nodes:20+2*n_nodes] /= np.sum(x0_f360m[17+2*n_nodes:20+2*n_nodes])
x0_f360m[21+3*n_nodes:24+3*n_nodes] /= np.sum(x0_f360m[21+3*n_nodes:24+3*n_nodes])



In [7]:
#Swap the entries of disk 3 and 4 in the x0_f300m and x0_f360m arrays
x0_f300m[2:12],x0_f300m[12:22] = x0_f300m[12:22],x0_f300m[2:12].copy()
x0_f360m[2:12],x0_f360m[12:22] = x0_f360m[12:22],x0_f360m[2:12].copy()

In [8]:
#Reformat common_parameters as a dictionary explicitly written out so that I could delete the above lines of code: 
common_parameters = {"Inclination": [r"$i$",[inc]],
                     "Position Angle":[r"$PA$", [pa]], 
                     "Roll 1 Center F300M": [r"[$x_{r1}$, $y_{r1}$]",[f300m_roll1_centers[1],f300m_roll1_centers[0]]],
                     "Roll 2 Center F300M": [r"[$x_{r2}$, $y_{r2}$]",[f300m_roll2_centers[1],f300m_roll2_centers[0]]],
                     "Roll 1 Center F360M": [r"[$x_{r1}$, $y_{r1}$]",[f360m_roll1_centers[1],f360m_roll1_centers[0]]],
                     "Roll 2 Center F360M": [r"[$x_{r2}$, $y_{r2}$]",[f360m_roll2_centers[1],f360m_roll2_centers[0]]],
                     }

#Disk Parameters names and symbols: 
disk_parameters = [["Flux Scaling", r"$n_0$"],
                   ["Disk Central Radius", r"$r_c$"],
                   ["Inner Power Law Index", r"$\alpha_{in}$"],
                   ["Outer Power Law Index", r"$\alpha_{out}$"],         
                   ]

disk_multi_parameters = [
                ["Scattering parameters", "[$g_1$, $g_2$, $g_3$]"],
                ["Scattering parameter weights", "[$w_1$, $w_2$, $w_3$]"],
]



In [9]:
# Start the LaTeX table
latex_table = """
\\begin{table*}[htbp]
\\centering
\\scriptsize
\\caption{Deluxe Parameter Table}
\\begin{tabular}{@{}lccccc@{}}
\\toprule
Parameter Name & Symbol(s) & Disk 1 & Disk 2 & Disk 3 & Disk 4 \\\\ \\midrule
"""
## In the middle of the table 
latex_table += f"\\multicolumn{{6}}{{c}}{{\\textbf{{Common Parameters}}}} \\\\\n"
latex_table += f"\\midrule\n"

for param in common_parameters:
    # Check if common_parameters[param][1] has two elements
    if  len(common_parameters[param][1]) == 2:
        # Format with square brackets and separated by a comma
        values_formatted = f"[{common_parameters[param][1][0]:.2f}, {common_parameters[param][1][1]:.2f}]"
    else:
        # It's a single value, format as before
        values_formatted = f"{common_parameters[param][1][0]:.2f}"
    
    # Add to the LaTeX table, centered
    latex_table += f"{param} & {common_parameters[param][0]} & \\multicolumn{{4}}{{c}}{{{values_formatted}}} \\\\\n"

#Now for the disk parameters: 

## In the middle of the table 
latex_table += f"\\multicolumn{{6}}{{c}}{{\\textbf{{Individual Disk Parameters}}}} \\\\\n"
latex_table += f"\\midrule\n"


for i,param in enumerate(disk_parameters):
    # Add the parameter name and symbol
    latex_table += f"{param[0]} & {param[1]} & "
    # Add the disk 1 values
    latex_table += f"{x0_f300m[2:6+n_nodes][i]:.2f} & "
    # Add the disk 2 values
    latex_table += f"{x0_f300m[6+n_nodes:10+2*n_nodes][i]:.2f} & "
    # Add the disk 3 values
    latex_table += f"{x0_f300m[10+2*n_nodes:14+3*n_nodes][i]:.2f} & "
    # Add the disk 4 values
    latex_table += f"{x0_f300m[14+3*n_nodes:18+4*n_nodes][i]:.2f} \\\\\n"
    
#Now for the disk multi parameters:
for i, param in enumerate(disk_multi_parameters):
    # Add the parameter name and symbol
    latex_table += f"{param[0]} & {param[1]} & "
    #Add the disk1 parameters
    values_formatted = f"[{x0_f300m[2:6+n_nodes][4+i*3]:.2f}, {x0_f300m[2:6+n_nodes][5+i*3]:.2f},{x0_f300m[2:6+n_nodes][6+i*3]:.2f}] & "
    #Add the disk2 parameters
    values_formatted += f"[{x0_f300m[6+n_nodes:10+2*n_nodes][4+i*3]:.2f}, {x0_f300m[6+n_nodes:10+2*n_nodes][5+i*3]:.2f},{x0_f300m[6+n_nodes:10+2*n_nodes][6+i*3]:.2f}] & "
    #Add the disk3 parameters
    values_formatted += f"[{x0_f300m[10+2*n_nodes:14+3*n_nodes][4+i*3]:.2f}, {x0_f300m[10+2*n_nodes:14+3*n_nodes][5+i*3]:.2f},{x0_f300m[10+2*n_nodes:14+3*n_nodes][6+i*3]:.2f}] & "
    #Add the disk4 parameters
    values_formatted += f"[{x0_f300m[14+3*n_nodes:18+4*n_nodes][4+i*3]:.2f}, {x0_f300m[14+3*n_nodes:18+4*n_nodes][5+i*3]:.2f},{x0_f300m[14+3*n_nodes:18+4*n_nodes][6+i*3]:.2f}] \\\\\n"
    latex_table += values_formatted

    


    
    

# Close the LaTeX table
latex_table += """
\\bottomrule
\\end{tabular}
\\end{table*}
"""

# Print or save the LaTeX code
print(latex_table)


\begin{table*}[htbp]
\centering
\scriptsize
\caption{Deluxe Parameter Table}
\begin{tabular}{@{}lccccc@{}}
\toprule
Parameter Name & Symbol(s) & Disk 1 & Disk 2 & Disk 3 & Disk 4 \\ \midrule
\multicolumn{6}{c}{\textbf{Common Parameters}} \\
\midrule
Inclination & $i$ & \multicolumn{4}{c}{58.37} \\
Position Angle & $PA$ & \multicolumn{4}{c}{-103.60} \\
Roll 1 Center F300M & [$x_{r1}$, $y_{r1}$] & \multicolumn{4}{c}{[146.94, 172.92]} \\
Roll 2 Center F300M & [$x_{r2}$, $y_{r2}$] & \multicolumn{4}{c}{[147.25, 172.71]} \\
Roll 1 Center F360M & [$x_{r1}$, $y_{r1}$] & \multicolumn{4}{c}{[147.25, 173.01]} \\
Roll 2 Center F360M & [$x_{r2}$, $y_{r2}$] & \multicolumn{4}{c}{[147.53, 172.82]} \\
\multicolumn{6}{c}{\textbf{Individual Disk Parameters}} \\
\midrule
Flux Scaling & $n_0$ & 170.73 & 48003.33 & 193.82 & 448.32 \\
Disk Central Radius & $r_c$ & 31.71 & 31.04 & 297.71 & 223.01 \\
Inner Power Law Index & $\alpha_{in}$ & 6.50 & 2.57 & 4.41 & 6.68 \\
Outer Power Law Index & $\alpha_{out}$ & 

The way the model fitting converged, ring 3 has a larger radius than ring 4, so I'm going to manually swap the table entries

\begin{table*}[htbp]
\centering
\scriptsize
\caption{Deluxe Parameter Table}
\begin{tabular}{@{}lccccc@{}}
\toprule
Parameter Name & Symbol(s) & Disk 1 & Disk 2 & Disk 3 & Disk 4 \\ \midrule
\multicolumn{6}{c}{\textbf{Common Parameters}} \\
\midrule
Inclination & $i$ & \multicolumn{4}{c}{58.37} \\
Position Angle & $PA$ & \multicolumn{4}{c}{-103.60} \\
Roll 1 Center F300M & [$x_{r1}$, $y_{r1}$] & \multicolumn{4}{c}{[146.94, 172.92]} \\
Roll 2 Center F300M & [$x_{r2}$, $y_{r2}$] & \multicolumn{4}{c}{[147.25, 172.71]} \\
Roll 1 Center F360M & [$x_{r1}$, $y_{r1}$] & \multicolumn{4}{c}{[147.25, 173.01]} \\
Roll 2 Center F360M & [$x_{r2}$, $y_{r2}$] & \multicolumn{4}{c}{[147.53, 172.82]} \\
\multicolumn{6}{c}{\textbf{Individual Disk Parameters}} \\
\midrule
Flux Scaling & $n_0$ & 170.73 & 48003.33 & 448.32 & 193.82  \\
Disk Central Radius & $r_c$ & 31.71 & 31.04 & 223.01 & 297.71  \\
Inner Power Law Index & $\alpha_{in}$ & 6.50 & 2.57 & 6.68 & 4.41  \\
Outer Power Law Index & $\alpha_{out}$ & -0.41 & -1.29 & -3.41 & -0.77  \\
Scattering parameters & [$g_1$, $g_2$, $g_3$] & [0.47, -0.70,0.44] & [0.84, 0.64,-0.90] & [0.61, -0.63,0.06] & [0.02, -0.19,0.80]  \\
Scattering parameter weights & [$w_1$, $w_2$, $w_3$] & [0.47, 0.38,0.14] & [0.00, 0.00,1.00] & [0.60, 0.40,0.00] & [0.47, 0.11,0.42] \\

\bottomrule
\end{tabular}
\end{table*}